In [ ]:
from alloskin.cli import load_config
from alloskin.io.readers import MDAnalysisReader
from alloskin.features.extraction import FeatureExtractor
from alloskin.pipeline.builder import DatasetBuilder
from alloskin.analysis.static import StaticReportersRF
from alloskin.analysis.qubo import QUBOSet

config = load_config(config_file="config/residue_selection.yaml")
residue_selections = config.get('residue_selections')

reader = MDAnalysisReader()
extractor = FeatureExtractor(residue_selections)
builder = DatasetBuilder(reader, extractor)

active_traj_file = "/storage_common/angiod/A2A/vdamore/FAN.5000.xtc"
active_topo_file = "/storage_common/angiod/A2A/vdamore/FAN.pdb"
inactive_traj_file = "/storage_common/angiod/A2A/vdamore/INapo.5000.xtc"
inactive_topo_file = "/storage_common/angiod/A2A/vdamore/INapo.pdb"

static_data = builder.prepare_static_analysis_data(
    active_traj_file,
    active_topo_file,
    inactive_traj_file,
    inactive_topo_file,
    "::100", "::100"
)

In [ ]:
# STATIC ANALYSIS

analyzer = StaticReportersRF()
results = analyzer.run(static_data, num_workers=64)
print("\nFinal Static Results (Sorted by best reporter, lowest II):")
results

In [ ]:
# QUBO ANALYSIS

print("\n--- Preparing Data for QUBO Analysis ---")

qubo_kwargs = {
    "active_topo_file": active_topo_file,
    "target_selection_string": "resid 10 20 30 40 50",
    "lambda_redundancy": 1.0,
    "num_solutions": 5,
    "qubo_cv_folds": 3,
    "qubo_n_estimators": 50
}

print("--- Running Optimal Predictive Set (QUBO) ---")
analyzer = QUBOSet()
results = analyzer.run(
    static_data, 
    num_workers=32,
    **qubo_kwargs
)

print("\n--- Final QUBO Results ---")
results